In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import colorcet as cc
import cartopy
import cartopy.crs as ccrs
import csv
import copy as cp
from scipy.stats import pearsonr, linregress
from scipy.signal import detrend
from scipy.io import loadmat
from matplotlib import cm, colors
from numba import jit, njit, vectorize

In [2]:
#The interval of time we're looking at (1980 to 2019)
startyear = 1980
endyear = 2019
timeseries = np.arange(startyear, endyear + 1, 1) #yearly
timeseries_cont = np.arange(startyear, endyear + 1, 1/12) #monthly

#dictionary of months
monthsnum = np.arange(1,13,1)
monthsind = np.arange(0,12,1)
monthsstr = ['01','02','03','04','05','06','07','08','09','10','11','12']
monthsnam = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
months = {1:'January',2:'February',3:'March',
          4:'April',5:'May',6:'June',7:'July',8:'August',
          9:'September',10:'October',11:'November',12:'December'}
names = ['NSIDC','Had2CIS','CANCM4i','GEM_NEMO','COMBINED']
ensembles = np.arange(0,10,1)

#colours for graphing
col = ['navy','mediumblue','mediumslateblue','violet',
       'maroon','red','orange','gold','lime',
       'forestgreen','purple','darkturquoise']

#regions
PanAntarctic = [0,360]
Weddell = [300,20]
Indian = [20,90]
WestPacific = [90,160]
Ross = [160,230]
Amundsen = [230,300]

In [3]:
#function to calculate sea ice extent from sea ice fraction/concentration

@njit
def calculate_SIE (SIC_array, latdat, londat, region):
    '''
    SIC_array: 2D array of SIC values in the form of [lat][lon]
    lat: 1D array of all latitudes
    lon: 1D array of all longitudes
    '''
    
    #Will keep track of sea ice extent
    SIE_counter = 0

    #iterate over all latitudes and longitudes
    for i in range(len(latdat)): 
        for j in range(len(londat)):
            
            #only consider SOUTHERN hemisphere
            if latdat[i] < 0:
                
                #only consider a certain region
                if region[0] > region[1]:
                    if londat[j] >= region[0] or londat[j] <= region[1]:

                        SIC = SIC_array[i][j]

                        if SIC >= 0.15: 
                            SIE_counter += (111.12**2) * np.abs(np.cos(latdat[i]*(np.pi/180)))
                            #km^2 area of each 1x1deg grid cell near south pole
                else:
                    if londat[j] >= region[0] and londat[j] <= region[1]:

                        SIC = SIC_array[i][j]

                        if SIC >= 0.15: 
                            SIE_counter += (111.12**2) * np.abs(np.cos(latdat[i]*(np.pi/180)))
                            #km^2 area of each 1x1deg grid cell near south pole
                            
    return SIE_counter/1e6



#function to write data to .txt file

def writedat(yearlydata, filename, loc = "Region_PanAntarcticSIE"):
    '''
    yearlydata: (dictionary) the data to be written to file, in the form of a dictionary
    filename: (string) the name of the file, excluding the .txt
    '''

    #First, create file
    #x is to create file
    #w is to rewrite file. If file doesn't exist, create file.
    #a is to append to file. If file doesn't exist, create file.
    #r is to read file

    f = open(loc + "/" + str(filename) + ".txt", "w")

    #go year by year
    for year in yearlydata.keys():

        f.write(str(year) + " ")

        #SIE month by month for the above year
        for val in yearlydata[year]:

            f.write(str(val) + " ")

        f.write("\n")

    f.close()
    
    
    
#function to retrieve data from .txt file

def retrievedat (filename, loc = "Region_PanAntarcticSIE"):
    '''
    retrieve data from a .txt file in a specific format
    '''
    
    #dictionary full of data to be returned
    datadict = {}
    
    f = open(loc + "/" + str(filename) + ".txt", "r")
    
    #go line by line
    for line in f:
        
        #split line into an array
        line.replace('\n', '')
        linearray = line.split()
        
        year = np.int64(linearray[0])
        dat = linearray[1:]
        
        #convert nones to None and values to floats
        for i,d in enumerate(dat):
            if d == 'None':
                dat[i] = np.nan
            else:
                dat[i] = np.float64(d)
        
        datadict[year] = dat
    
    f.close()
    
    return datadict



#function to calculate average of array w/ None types

def average (array):
    
    net = 0
    count = 0
    
    for dat in array:
        if dat is not None and np.isnan(dat) == False:
            net += dat
            count += 1
            
    return net/count


#function to calculate average between CANCM4 and GEM_NEMO

def model_average (model1, model2, N=2):
    
    dictaveraged = {}
    
    for year in timeseries:
        
        dictaveraged[year]= [np.nan]*12
        
        for leadtime in monthsind:
            
            net = model1[year][leadtime] + model2[year][leadtime]
            ave = net/N #average
            
            dictaveraged[year][leadtime] = ave
    
    return dictaveraged


#function to retrieve all data pertaining to a specific month of the year

def DataForMonth (dataset, month, initmonth = 1):
    
    #Determine what type the input is in (e.g. array or dictionary)
    if type(dataset) == dict:
        dataset1D = Dict1D(dataset)
    else:
        dataset1D = dataset
            
    #Now pick out all the data corresponding to the specified month
    #e.g. if month = 1 and initmonth = 1, then it should pick out index 0, index 12, index 24, etc...
    
    index = (month - 1) - (initmonth - 1)
    datasetmonth = []

    if month < initmonth:
        index += 12
    while index < len(dataset1D):
        datasetmonth.append(dataset1D[index])
        index += 12 #jump forward 12 months
            
    return datasetmonth


#Put a dictionary of data into a 1D array

def Dict1D (dataset):
    
    #put all the data into a 1D array
    
    dataset1D = []
    for year in dataset.keys():
        for im in monthsind:
            dataset1D.append(dataset[year][im])
    
    return dataset1D


#Quickplot

def QuickPlot (datadict, colour = "Navy"):
    
    fig, axis = plt.subplots(1,1,figsize=(8,6))
    
    xaxis = timeseries_cont
    yaxis = Dict1D(datadict)
    
    axis.plot(xaxis, yaxis, color = colour)
    axis.grid()
    
    fig.tight_layout()

In [1]:
# filename = "sf_chfp3b-hindcast_198101-202012_init_1x1_sithick.nc"
# file = nc.Dataset(filename,'r')
# SITfieldlist = file.variables['sithick'][:]
# file.close()

# aveSITdict = {}
# for y_i, year in enumerate(timeseries):
#     aveSITdict[year+1] = [[]]*12
    
#     for m_i in monthsind:
        
#         SITfield = SITfieldlist[12*y_i + m_i]
#         aveSIT = np.ma.mean(SITfield)
#         aveSITdict[year+1][m_i] = aveSIT
    
# writedat(aveSITdict, "initialSIT_canESM5", loc = "Region_WeddellSIE")
# print(aveSITdict)

In [4]:
def getCANCM4data (ensemble, region):

    #initmonth should be a string, e.g. '01'
    #ensemble should be a integer in [0,9]

    #set up dictionary
    CANCM4data = {}
    for year in timeseries:
        CANCM4data[year]= [np.nan]*12


    #Now to actually retrieve the data from the nc files
    
    for year in timeseries:
        for initmonth in monthsstr:

            #open file

            #"sic0_sic_daily_CCCma-CanCM4_NEW_1x1_grid_i19800101_1"

            startstr = "CANCM4init\\" + initmonth + "\\sic0_sic_daily_CCCma-CanCM4_NEW_1x1_grid_i"
            endstr = str(year) + initmonth + '01_' + str(ensemble+1) + '.nc'
            rawfile = startstr + endstr

            file = nc.Dataset(rawfile, 'r')

            #(only need to get latitude and longitude once)
            if year == startyear and initmonth == '01':
                latitudes = file.variables['latitude'][:]
                longitudes = file.variables['longitude'][:]

            SIC = np.array(file.variables['sic'][:][0])
            SIC = np.where(SIC > 1, 0, SIC)
            SIE_val = calculate_SIE(SIC, latitudes, longitudes, region)
            CANCM4data[year][int(initmonth)-1] = SIE_val

            #close file
            file.close()
        
        
    return CANCM4data

In [19]:
for ens in ensembles:
    print(ens)
        
    SIEdata = getCANCM4data(ens, region=Weddell)
        
    filename = "CANCM4initE" + str(ens)
        
    writedat(SIEdata, filename, loc = "Region_WeddellSIE")

0


<ipython-input-18-da51deca8217>:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  latitudes = file.variables['latitude'][:]
<ipython-input-18-da51deca8217>:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  longitudes = file.variables['longitude'][:]
<ipython-input-18-da51deca8217>:32: DeprecationWarning: `np.bool` is a deprecated alias for the buil

1


<ipython-input-18-da51deca8217>:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  latitudes = file.variables['latitude'][:]
<ipython-input-18-da51deca8217>:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  longitudes = file.variables['longitude'][:]


2
3
4
5
6
7
8
9


In [20]:
def getGEMNEMOdata (ensemble, region):

    #initmonth should be a string, e.g. '01'
    #ensemble should be a integer in [0,9]

    #set up dictionary
    GEMNEMOdata = {}
    for year in timeseries:
        GEMNEMOdata[year]= [np.nan]*12


    #Now to actually retrieve the data from the nc files
    
    for year in timeseries:
        for initmonth in monthsstr:

            #open file

            #"sic0_sic_daily_CCCma-CanCM4_NEW_1x1_grid_i19800101_1"

            startstr = "GEMNEMOinit\\" + initmonth + "\\sic0_sic_daily_CCCma-GEM_NEMO_1x1_grid_i"
            endstr = str(year) + initmonth + '01_' + str(ensemble+1) + '.nc'
            rawfile = startstr + endstr

            file = nc.Dataset(rawfile, 'r')

            #(only need to get latitude and longitude once)
            if year == startyear and initmonth == '01':
                latitudes = file.variables['latitude'][:]
                longitudes = file.variables['longitude'][:]

            SIC = np.array(file.variables['sic'][:][0])
            SIC = np.where(SIC > 1, 0, SIC)
            SIE_val = calculate_SIE(SIC, latitudes, longitudes, region)
            GEMNEMOdata[year][int(initmonth)-1] = SIE_val

            #close file
            file.close()
        
        
    return GEMNEMOdata

In [21]:
for ens in ensembles:
    print(ens)
        
    SIEdata = getGEMNEMOdata(ens, region=Weddell)
        
    filename = "GEMNEMOinitE" + str(ens)
        
    writedat(SIEdata, filename, loc = "Region_WeddellSIE")

0


<ipython-input-20-863ee0256016>:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  latitudes = file.variables['latitude'][:]
<ipython-input-20-863ee0256016>:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  longitudes = file.variables['longitude'][:]
<ipython-input-20-863ee0256016>:32: DeprecationWarning: `np.bool` is a deprecated alias for the buil

1
2
3
4
5
6
7
8
9


# Average over Ensembles

In [22]:
#CANCM4

regionname = "Weddell"

#retrieve necessary data from each ensemble
alldata = []

for ens in ensembles:
    
    filename = 'CANCM4initE' + str(ens)
    alldata.append(retrievedat(filename, loc="Region_"+regionname+"SIE"))

    
    
#calculate the mean across ensembles
averages = {}
for year in timeseries:
    averages[year] = [np.nan]*12
    
for year in timeseries:
    for month in monthsind:
        vals = []
        for dats in alldata:
            vals.append(dats[year][month])
        averages[year][month] = average(vals)
        
#write to file
writedat(averages, "CANCM4initAv",loc="Region_"+regionname+"SIE")

In [23]:
#GEMNEMO

#retrieve necessary data from each ensemble
alldata = []

for ens in ensembles:
    
    filename = 'GEMNEMOinitE' + str(ens)
    alldata.append(retrievedat(filename, loc="Region_"+regionname+"SIE"))

    
    
#calculate the mean across ensembles
averages = {}
for year in timeseries:
    averages[year] = [np.nan]*12
    
for year in timeseries:
    for month in monthsind:
        vals = []
        for dats in alldata:
            vals.append(dats[year][month])
        averages[year][month] = average(vals)
        
#write to file
writedat(averages, "GEMNEMOinitAv",loc="Region_"+regionname+"SIE")